In [45]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
plt.rc('font', family='Verdana')

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import GridSearchCV

In [46]:
data = DataFrame.from_csv('data/processed_data.csv', encoding='utf-8')
data.head()

,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,...,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED,AGE_CATEGORY,WITHOUT_DELAY,REG_ADDRESS_PROVINCE_FREQ,FACT_ADDRESS_PROVINCE_FREQ,SOLVENCY
0,0,49,0,0,0,0,0,0,0,0,...,2,1,1580.000000,1580.0,0.0,0,0,253,250,3.750000
1,0,32,0,0,0,1,1,1,0,0,...,1,1,4020.000000,4020.0,0.0,1,0,127,127,3.325635
2,0,52,0,0,0,2,2,2,0,1,...,0,0,0.000000,0.0,0.0,2,1,202,203,3.260279
3,0,39,0,0,0,3,0,3,0,2,...,3,1,1589.923333,1590.0,0.0,1,0,407,404,17.664058
4,0,30,0,0,1,4,2,1,0,3,...,2,1,1152.150000,2230.0,0.0,3,0,127,127,6.548431


In [47]:
data.shape

(15223, 55)

In [48]:
X = data.drop('TARGET', axis=1)
y = data['TARGET']

In [ ]:
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
trees_grid = [5, 10, 25, 50, 100, 150, 200, 250, 300]

for ntrees in trees_grid:
    rf = RandomForestClassifier(n_estimators=ntrees, random_state=42, n_jobs=-1)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in strat.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        rf.fit(X_train, y_train)
        temp_train_acc.append(rf.score(X_train, y_train))
        temp_test_acc.append(rf.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)

train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Наилучшее значение правильности перекрестной проверки равно {:.2f}% и достигается при {} деревьях".format(max(test_acc.mean(axis=1))*100, 
                                                        trees_grid[np.argmax(test_acc.mean(axis=1))]))

In [ ]:
plt.style.use('ggplot')

fig, ax = plt.subplots(figsize=(16, 10))
ax.plot(trees_grid, train_acc.mean(axis=1), alpha=0.5, color='blue', 
        label='обучение')
ax.plot(trees_grid, test_acc.mean(axis=1), alpha=0.5, color='red', 
        label='перекрестная проверка')
ax.fill_between(trees_grid, test_acc.mean(axis=1) - test_acc.std(axis=1), test_acc.mean(axis=1) + test_acc.std(axis=1), color='#888888', alpha=0.4)
ax.fill_between(trees_grid, test_acc.mean(axis=1) - 2*test_acc.std(axis=1), test_acc.mean(axis=1) + 2*test_acc.std(axis=1), color='#888888', alpha=0.2)
ax.legend(loc='best')
ax.set_ylim([0.85,1.01])
ax.set_ylabel("Правильность")
ax.set_xlabel("Количество деревьев")

In [ ]:
n_trees = 75

In [ ]:
strat = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

X_trainval, X_test, y_trainval, y_test = train_test_split(data.drop('TARGET', axis=1), data['TARGET'],
                                                          test_size=0.3, random_state=42)
print("Размер обучающей выборки: {} размер тестовой выборки: {}".format(X_trainval.shape[0], X_test.shape[0]))

best_score = 0

for max_depth in [10, 12, 14, 16, 18, 20, 22]:
    for max_features in [3, 6, 9, 12, 15, 18, 21]:
        # для каждой комбинации параметров обучаем случайный лес
        rf = RandomForestClassifier(max_depth=max_depth, max_features=max_features,  
                                    n_estimators=75, n_jobs=-1, random_state=42)
        # выполняем перекрестную проверку        
        scores = cross_val_score(rf, X_trainval, y_trainval, scoring='roc_auc', cv=strat)
        # вычисляем среднее значение AUC перекрестной проверки 
        auc_score = np.mean(scores)
        # если получаем лучшее значение AUC, сохраняем его и параметры
        if auc_score > best_score:
            best_score = auc_score
            best_parameters = {'max_features': max_features, 'max_depth': max_depth}
# строим модель с параметрами, давшими наилучшее значение AUC на обучающей выборке, 
# и проверяем на тестовой выборке
rf_best = RandomForestClassifier(**best_parameters, n_estimators=300, random_state=42)
rf_best.fit(X_trainval, y_trainval)
test_score = roc_auc_score(y_test, rf_best.predict_proba(X_test)[:, 1])
print("Лучшее усредненное значение AUC cv: {:.2f}".format(best_score))
print("Наилучшие значения параметров: ", best_parameters)
print("AUC модели с наилучшими параметрами на тестовой выборке: {:.2f}".format(test_score))

In [ ]:
param_grid = {'max_features': [3, 4, 5, 6, 7, 8, 9, 12, 15, 18, 21], 'max_depth': [6, 8, 10, 12, 14, 16, 18, 20, 22]}

In [ ]:
rf = RandomForestClassifier(n_estimators=n_trees, random_state=42)
grid_search = GridSearchCV(rf, param_grid, n_jobs=-1, cv=strat)
grid_search.fit(X_trainval, y_trainval)
test_score = roc_auc_score(y_test, grid_search.predict(x_test))
print("AUC на тестовой выборке: {:.2f}".format(test_score))
print("Наилучшие значения параметров: {}".format(grid_search.best_params_))

In [6]:
logreg = LogisticRegression()

In [7]:
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
logreg.score(X_test, y_test)

0.87861271676300579

In [13]:
logreg.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [15]:
# Из модуля sklearn.metrics импортируем функцию roc_auc_score
from sklearn.metrics import roc_auc_score
# Вычисляем значение AUC, передав функции roc_curve в качестве аргументов
# фактические значения зависимой переменной, записанные в y_control, 
# и вероятности положительного класса, вычисленные с помощью метода predict_proba
rf_auc = roc_auc_score(y_test, logreg.predict(X_test))
# Печатаем значение AUC
print("Значение AUC для контрольной выборки: {:.3f}".format(rf_auc))

Значение AUC для контрольной выборки: 0.500
